In [8]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import plotly.express as px
import re

In [9]:
# read in csv
url="https://opendata.ecdc.europa.eu/covid19/testing/csv/data.csv"
covid_data = pd.read_csv(url)

In [10]:
# define countries for subsetting
countries = ["Denmark", "Germany", "Italy", "Spain", "Sweden"]

In [11]:
def find_first_day_of_week(dates_string):
    # The values are %Y = year; %W = week; %w = weekday + 1= monday
    dates_formated = pd.to_datetime(dates_string +'-1', format="%Y-W%W-%w")
    
    return dates_formated

In [14]:
def subset_covid_data(covid_data, countries): # assuming population
    
    # subset with countries
    covid_data_subset = covid_data[covid_data["country"].isin(countries)].copy()
    
    # format the input date coulumn as datetime
    covid_data_subset["dates"] = find_first_day_of_week(covid_data_subset["year_week"])
    
    # make column as mont and year for plotting
    covid_data_subset["months"] = covid_data_subset['dates'].dt.strftime('%Y-%m').astype(str)
    
    # reset index and resample the data on dates
    covid_data_subset.set_index('dates', inplace=True)
    covid_summary = covid_data_subset.groupby(['country', 'months']).resample('M').sum().reset_index()
    
    return covid_summary

In [15]:
covid_subset_data = subset_covid_data(covid_data, countries)
fig = px.bar(covid_subset_data, x='months', y='tests_done', color='country')
fig.show()